# Create dataset from public data at national level

This file is part of the replication code for: 

Offshore Profit Shifting and Aggregate Measurement: Balance of Payments, Foreign Investment, Productivity, and the Labor Share by Guvenen, Mataloni Jr., Rassier, and Ruhl. 

This version: February 21, 2022

This file processes publicly available data files (located in `1-raw-data`) to create `aggregate.csv` which is saved in the `3-intermediate-files` folder.

In [2]:
import pandas as pd
import numpy as np

## Aggregate data from NIPA

BEA offers the entire NIPA tables as a flat file. It can be downloaded [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=19&step=2#reqid=19&step=2&isuri=1&1921=survey).

In [3]:
bea = pd.read_csv('../1-raw-data/SeriesRegister.txt')
nipa = pd.read_csv('../1-raw-data/NipaDataA.txt')

# Gross value added in business sector (nominal) = A195RC
# Gross value added in business sector (real) = A195RX
# Gross national product (nominal) = A001RC
# Gross domestic product (nominal) = A191RC
# Gross domestic income (nominal) = A261RC
# Coporate business (net income) (nominal) = A439RC
# Compensation of employees (nominal) = A442RC
# Consumption of fixed capital, corporate business (nominal) = A438RC
# Coporate profits w/ inventory valuation and cca, domestic industries = A445RC

series = {'A195RC':'gvabusn', 'A195RX':'gvabusr', 'A001RC':'gnpn', 'A191RC':'gdpn',
         'A261RC':'gdin', 'A439RC':'corpincn', 'A442RC':'compn', 'A438RC':'cfcn', 'A445RC':'dom_profitn'}

In [4]:
public = nipa[nipa['%SeriesCode'].isin(list(series.keys()))]
public.columns = ['series', 'period', 'value']
public = public.set_index(['series', 'period'])
public = public.unstack(level='series')
public.columns = public.columns.droplevel(level=0)
public = public.rename(columns=series)

for c in public.columns:
    public[c] = public[c].str.replace(',','').astype(int)
    
public = public.loc[1982:2017].copy()

# Express in billions
public = public/1000
public['gva_deflator'] = public['gvabusn']/public['gvabusr']

public.head(1)

series,gnpn,gdpn,gvabusn,gvabusr,gdin,cfcn,corpincn,compn,dom_profitn,gva_deflator
period,,,,,,,,,,
1982,3380.299,3343.789,2568.636,4649.957,3334.953,272.52,1745.861,1275.114,197.27,0.5524


## International transaction accounts

Tables are available to download from [here](https://apps.bea.gov/iTable/iTable.cfm?ReqID=19&step=4&isuri=1&1921=flatfiles&3Place=Y#reqid=19&step=4&isuri=1&1921=flatfiles )

### ITA Table 1.1 (data in millions)

In [7]:
ita11 = pd.read_excel('../1-raw-data/Ita_T1.1.xls', header=5, nrows=44, na_values=['n.a.', '.....', ' ', '\xa0'])
temp = ita11.columns.to_list()
temp[1] = 'Series'
ita11.columns = temp
ita11['Series'] = ita11['Line']+'-'+ita11['Series'].str.strip()
ita11 = ita11.drop('Line', axis=1)
ita11.set_index(['Series'], inplace=True)
ita11 = ita11.transpose()
ita11.index = ita11.index.astype(int)
ita11.head(1)

Series,NaN,1-Exports of goods and services and income receipts (credits),2-Exports of goods and services,3-Goods,4-Services,5-Primary income receipts,6-Investment income,7-Compensation of employees,8-Secondary income (current transfer) receipts /1/,9-Imports of goods and services and income payments (debits),...,NaN,30-Balance on current account (line 1 less line 9) /4/,31-Balance on goods and services (line 2 less line 10),32-Balance on goods (line 3 less line 11),33-Balance on services (line 4 less line 12),34-Balance on primary income (line 5 less line 13),35-Balance on secondary income (line 8 less line 16),36-Balance on capital account (line 17 less line 18) /4/,37-Net lending (+) or net borrowing (-) from current- and capital-account transactions (line 30 plus line 36) /5/,38-Net lending (+) or net borrowing (-) from financial-account transactions (line 19 less line 24 plus line 28) /5/
1960,NaN,30555.0,25939.0,19650.0,6289.0,4616.0,4616.0,NaN,NaN,27732.0,...,NaN,2825.0,3508.0,4892.0,-1385.0,3378.0,-4061.0,NaN,2825.0,1805.0


In [8]:
ita11_cols = {'31-Balance on goods and services (line 2 less line 10)': 'tradebal',
             '32-Balance on goods (line 3 less line 11)': 'goodbal',
             '33-Balance on services (line 4 less line 12)':'servbal'
             }
public = pd.merge(left=public, right=ita11[ita11_cols.keys()], left_index=True, right_index=True)

# Express in billions
public[list(ita11_cols.keys())] = public[list(ita11_cols.keys())]/1000  
public = public.rename(columns=ita11_cols)

### ITA Table 4.2 (data in millions)

In [9]:
ita42 = pd.read_excel('../1-raw-data/Ita_T4.2.xls', header=5, skipfooter=9)
temp = ita42.columns.to_list()
temp[1] = 'Series'
ita42.columns = temp
ita42['Series'] = ita42['Line']+'-'+ita42['Series'].str.strip()
ita42 = ita42.drop('Line', axis=1)
ita42 = ita42.set_index(['Series'])
ita42 = ita42.stack().unstack(['Series'])
ita42.index = ita42.index.astype(int)
ita42.head(1)

Series,-Receipts,"1-Direct investment income on assets, asset/liability basis (table 4.1, line 3) /1/",2-Income on equity,3-Dividends and withdrawals,4-Reinvested earnings,5-Interest,6-U.S. parents' receipts,7-U.S. affiliates' receipts,8-Less: Adjustments to convert to directional basis,9-U.S. parents' interest payments,...,62-Income on equity without current-cost adjustment (line 49 less line 53),63-Manufacturing,64-Wholesale trade,65-Finance (including depository institutions) and insurance,66-Other,"67-Interest, net payments",68-Manufacturing,69-Wholesale trade,70-Finance (including depository institutions) and insurance,71-Other
1982,,34243,31343,23058,8285,2900,2264,636,4774,4138,...,1126,n.a.,n.a.,n.a.,n.a.,2165,n.a.,n.a.,n.a.,n.a.


In [10]:
ita42_cols = {'2-Income on equity':'usdiaincn',
              '26-Income on equity without current-cost adjustment (line 12 less line 16)':'usdiaincn_wocca',
              '57-Direct investment income without current-cost adjustment on inward investment, directional basis (line 48 less line 53) /2/':'fdiusinc',
              '20-Direct investment income without current-cost adjustment on outward investment, directional basis (line 11 less line 16) /2/':'usdiainc'
             }

public = pd.merge(left=public, right=ita42[ita42_cols.keys()].astype(int), left_index=True, right_index=True)

# Express in billions
public[list(ita42_cols.keys())] = public[list(ita42_cols.keys())]/1000

public = public.rename(columns=ita42_cols)
public['income_adj_factor'] = public['usdiaincn']/public['usdiaincn_wocca']

## Oil prices

These data are from FRED (code = 'WTISPLC'). 

In [11]:
poil = pd.read_excel('../1-raw-data/WTISPLC.xls', header=10)
poil = poil.set_index(poil['observation_date'].dt.year)
poil = poil.rename(columns={'WTISPLC':'poil'})
poil.head(1)

,observation_date,poil
observation_date,,
1982,1982-01-01,33.635833


In [12]:
public = pd.merge(left=public, right=poil['poil'], left_index=True, right_index = True)
public.head(1)

,gnpn,gdpn,gvabusn,gvabusr,gdin,cfcn,corpincn,compn,dom_profitn,gva_deflator,tradebal,goodbal,servbal,usdiaincn,usdiaincn_wocca,fdiusinc,usdiainc,income_adj_factor,poil
1982,3380.299,3343.789,2568.636,4649.957,3334.953,272.52,1745.861,1275.114,197.27,0.5524,-24.156,-36.485,12.33,31.343,27.865,3.291,25.991,1.124816,33.635833


## IIP tables
Available to download [here](https://apps.bea.gov/iTable/bp_download_modern.cfm?pid=5).

In [13]:
iip21 = pd.read_excel('../1-raw-data/Iip_T2.1.xls', header=5, nrows=47, na_values=['n.a.', '.....', ' ', '\xa0'])
iip21['Series'] = iip21['Line'].astype(str)+'-'+iip21['Type of investment'].str.strip()
iip21 = iip21.drop(['Line', 'Type of investment'], axis=1)
iip21.set_index(['Series'], inplace=True)
iip21 = iip21.transpose()
iip21.index = iip21.index.astype(int)
iip21.head(1)

Series,nan-Assets / outward investment,"1.0-U.S. direct investment assets at market value, asset/liability basis (table 1.1, line 7) /1/",2.0-Equity,3.0-Debt instruments,4.0-U.S. parents' claims,5.0-U.S. affiliates' claims,6.0-Less: Adjustments to convert to directional basis,7.0-U.S. parents' liabilities,8.0-U.S. affiliates' claims,"9.0-Equals: Outward direct investment (U.S. direct investment abroad) at market value, directional basis /2/",...,"35.0-Direct investment assets at current cost, asset/liability basis /1/",36.0-Equity,37.0-Debt instruments,38.0-Less: Adjustments to convert to directional basis,"39.0-Equals: Outward direct investment at current cost, directional basis /2/","40.0-Direct investment liabilities at current cost, asset/liability basis /1/",41.0-Equity,42.0-Debt instruments,43.0-Less: Adjustments to convert to directional basis,"44.0-Equals: Inward direct investment at current cost, directional basis /2/"
1982,NaN,295981.0,227369.0,68612.0,59907.0,8705.0,69343.0,60638.0,8705.0,226638.0,...,443402.0,374790.0,68612.0,69343.0,374059.0,254185.0,153606.0,100579.0,69343.0,184842.0


In [14]:
iip21_cols = {'32.0-Equals: Inward direct investment at historical cost, directional basis':'fdiusstks',
              '15.0-Equals: Outward direct investment at historical cost, directional basis':'usdiastks'}

public = pd.merge(left=public, right=iip21.loc[1982:2016,iip21_cols.keys()], left_index=True, right_index=True, how='right')
# Express in billions
public[list(iip21_cols.keys())] = public[list(iip21_cols.keys())]/1000
public = public.rename(columns=iip21_cols)

## Direct investment income by country
Downloaded from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=2&step=1&isuri=1#reqid=2&step=1&isuri=1).

In [15]:
tax_havens = ['Ireland', 'Luxembourg', 'Netherlands', 'Switzerland', 'Liechtenstein', 'Costa Rica', 'Bahamas', 'Barbados',
              'United Kingdom Islands, Caribbean[2]', 'Bermuda', 'Hong Kong', 'Singapore']

usdiainc_cty = pd.read_excel('../1-raw-data/usdiainc_country.xls', header=6, nrows=253, 
                             na_values=['n.s.', '(*)', '(D)', '----', '---', '...', '--'])
usdiainc_cty = usdiainc_cty.rename(columns={'Unnamed: 0':'country'})
usdiainc_cty['country'] = usdiainc_cty['country'].str.strip()
usdiainc_cty = usdiainc_cty.set_index('country').transpose()
usdiainc_cty.index = usdiainc_cty.index.astype(int)
usdiainc_cty['usdiainc_havens'] = usdiainc_cty[tax_havens].sum(axis=1)
usdiainc_cty['usdiainc_nhavens'] = usdiainc_cty['All Countries Total'] - usdiainc_cty['usdiainc_havens']
public = pd.merge(left=public, right=usdiainc_cty[['usdiainc_havens', 'usdiainc_nhavens']], left_index=True, right_index=True)

# Express in billions
public[['usdiainc_havens', 'usdiainc_nhavens']] = public[['usdiainc_havens', 'usdiainc_nhavens']]/1000

## Direct investment position by country
Download from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=2&step=1&isuri=1#reqid=2&step=1&isuri=1).

In [16]:
usdia_cty = pd.read_excel('../1-raw-data/usdia_country.xls', header=6, nrows=253, 
                             na_values=['n.s.', '(*)', '(D)', '----', '---', '...', '--'])
usdia_cty = usdia_cty.rename(columns={'Unnamed: 0':'country'})
usdia_cty['country'] = usdia_cty['country'].str.strip()
usdia_cty = usdia_cty.set_index('country').transpose()
usdia_cty.index = usdia_cty.index.astype(int)
usdia_cty['usdia_havens'] = usdia_cty[tax_havens].sum(axis=1)
usdia_cty['usdia_nhavens'] = usdia_cty['All Countries Total'] - usdia_cty['usdia_havens']
usdia_cty[['usdia_havens', 'usdia_nhavens']].head()
public = pd.merge(left=public, right=usdia_cty[['usdia_havens', 'usdia_nhavens']], left_index=True, right_index=True)

# Express in billions
public[['usdia_havens', 'usdia_nhavens']] = public[['usdia_havens', 'usdia_nhavens']]/1000

## Total value added for MNE parents and affiliates

1. Parents from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=2&step=10&isuri=1&step1prompt1=1&step2prompt3=14&step1prompt2=2&step8prompt10a=94&step4prompt5=99&step3prompt4=6&step5prompt6=1,2&step7prompt8=40,41,42,43,48,49,52,55,56,58,60&step8prompt9a=45)
2. Affiliates from [here](https://apps.bea.gov/iTable/iTable.cfm?reqid=2&step=10&isuri=1&step1prompt1=1&step2prompt3=13&step1prompt2=2&step8prompt10a=94&step4prompt5=99&step3prompt4=6&step5prompt6=1,2&step7prompt8=40,41,42,43,48,49,52,55,56,58,60&step8prompt9a=45)

In [17]:
total_aff_va = pd.read_csv('../1-raw-data/va-total-affiliate.csv', header=5, nrows=1, index_col='Unnamed: 0').transpose()
total_par_va = pd.read_csv('../1-raw-data/va-total-parent.csv', header=5, nrows=1, index_col='Unnamed: 0').transpose()

total_aff_va.index = total_aff_va.index.astype(int)
total_par_va.index = total_par_va.index.astype(int)

# Express in billions
total_mne_va = (total_par_va+total_aff_va)/1000
total_mne_va.columns = ['total_mne_va']

total_par_va = total_par_va/1000
total_par_va.columns = ['total_mne_dom_va']

public = pd.concat([public, total_mne_va.loc[:2016], total_par_va.loc[:2016]], axis=1)
public.head(1)

,gnpn,gdpn,gvabusn,gvabusr,gdin,cfcn,corpincn,compn,dom_profitn,gva_deflator,...,income_adj_factor,poil,fdiusstks,usdiastks,usdiainc_havens,usdiainc_nhavens,usdia_havens,usdia_nhavens,total_mne_va,total_mne_dom_va
1982,3380.299,3343.789,2568.636,4649.957,3334.953,272.52,1745.861,1275.114,197.27,0.5524,...,1.124816,33.635833,124.677,207.752,8.046,16.782,43.917,163.835,NaN,NaN


## Employment data for United States

From BLS

1. Go here: https://data.bls.gov/cgi-bin/srgate
2. Enter: 'PRS84006033'
3. Choose "Annual data" and then hit retrieve. 
4. Download the xlsx file

In [19]:
hrs = pd.read_excel('../1-raw-data/bls-PRS84006033.xlsx', header=10, index_col='Year')
hrs = hrs.rename(columns={'Annual':'hrs'})
public = pd.merge(left=public, right=hrs, left_index=True, right_index=True)

/Users/danielfichmann/Library/Python/3.9/lib/python/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Write the file

Write the DataFrame to the `3 intermediate files` to be used in the analysis. 

In [20]:
public.index.name = 'year'
public.to_csv('../3-intermediate-files/aggregate.csv')
public

,gnpn,gdpn,gvabusn,gvabusr,gdin,cfcn,corpincn,compn,dom_profitn,gva_deflator,...,poil,fdiusstks,usdiastks,usdiainc_havens,usdiainc_nhavens,usdia_havens,usdia_nhavens,total_mne_va,total_mne_dom_va,hrs
year,,,,,,,,,,,,,,,,,,,,,
1982,3380.299,3343.789,2568.636,4649.957,3334.953,272.520,1745.861,1275.114,197.270,0.552400,...,33.635833,124.677,207.752,8.046,16.782,43.917,163.835,NaN,NaN,73.952
1983,3671.122,3634.038,2801.887,4896.401,3577.085,285.876,1886.483,1352.980,244.711,0.572234,...,30.395000,137.061,212.150,8.344,18.916,47.690,164.460,NaN,NaN,75.315
1984,4073.903,4037.613,3136.701,5330.710,3996.035,303.168,2131.464,1501.072,301.313,0.588421,...,29.275500,164.583,218.093,8.204,22.542,50.386,167.707,NaN,NaN,79.723
1985,4364.344,4338.979,3369.644,5579.285,4284.716,326.864,2274.439,1615.931,316.390,0.603956,...,27.972750,184.615,238.369,7.994,20.784,55.084,183.285,NaN,NaN,81.552
1986,4596.588,4579.631,3539.300,5781.967,4499.578,349.868,2356.695,1723.353,284.900,0.612127,...,15.040000,220.414,270.472,9.321,21.719,64.579,205.893,NaN,NaN,82.194
1987,4872.702,4855.215,3735.245,5989.477,4811.392,371.292,2523.622,1847.640,317.973,0.623635,...,19.161917,263.394,326.253,11.107,28.921,77.761,248.492,NaN,NaN,84.663
1988,5259.095,5236.438,4019.261,6246.006,5233.436,399.856,2742.499,2002.309,357.471,0.643493,...,15.959583,314.754,347.179,12.541,39.516,79.777,267.402,NaN,NaN,86.987
1989,5666.369,5641.580,4326.705,6485.121,5573.590,427.148,2892.834,2119.332,347.166,0.667174,...,19.590833,368.924,381.781,14.015,39.914,84.632,297.149,NaN,NaN,89.276
1990,5997.836,5963.144,4541.951,6588.974,5867.597,454.688,3012.451,2234.926,341.641,0.689326,...,24.492917,394.911,430.521,16.015,41.989,92.807,337.714,NaN,NaN,88.920
